# Embryoid Bodies 2018

> Embryoid Bodies 2018 dataset.

In [ ]:
#| default_exp datasets.embryoid

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os, math
import numpy as np, pandas as pd

import torch, torch.nn as nn, pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader, TensorDataset

import matplotlib as mpl, matplotlib.pyplot as plt, seaborn as sns

In [ ]:
#| export
from dataclasses import dataclass, field, KW_ONLY

from beartype.typing import Optional, Tuple, Union, TypeAlias, Dict, Any
from nptyping import NDArray, Float, Shape, Number as AnyNumber
from beartype import beartype

#| export
from littyping.core import Device

In [ ]:
#| export
from iza.utils import wrangle_kwargs_for_func, make_missing_dirs
from iza.static import SAMPLES

In [ ]:
#| export
from litds.abc.dfdm.base import set_dataset

from litds.core.dfds import (DataFrameDataset)
from litds.core.dfdm import (DataFrameDataModule)
from litds.time.base import (TimeDataset, TimeDataModule)

from litds.utils import (random_split_dataframe)

from litds.types import (
    Number, DataFrame, IndexLike, SeriesLike, GroupKey,  
    TrainValidTestSplit,
    XYArray, DistanceArray, LabelArray,    
    SequenceWithLength, SequencesWithLengths
)

In [ ]:
#| export
from kuut.core import (kuut, kstep)

In [ ]:
#| export
import requests, zipfile, io
import phate, scprep, pickle

## Constants

In [ ]:
#| export
MENDELY_URI = (
    'https://data.mendeley.com/public-files/'
    'datasets/v6n743h5ng/files/b1865840-e8df-4381-'
    '8866-b04d57309e1d/file_downloaded'
)

SINGLE_CELL_SUBDIR = 'scRNAseq'
# SAMPLES = 'samples'

TIMEPOINT_DIRS = 'T0_1A T2_3B T4_5C T6_7D T8_9E'.split()

HUMAN_FRIENDLY_TIMEPOINTS = 'Day 00-03,Day 06-09,Day 12-15,Day 18-21,Day 24-27'.split(',')

STEPS = [
    kstep(1, val=20, desc='download'),
    kstep(2, val=5, sub=0, desc='load_timepoints'),
    kstep(2, val=5, sub=1, desc='load_timepoints'),
    kstep(2, val=5, sub=2, desc='load_timepoints'),
    kstep(2, val=5, sub=3, desc='load_timepoints'),
    kstep(2, val=5, sub=4, desc='load_timepoints'),

    kstep(3, val=5, sub=0, desc='library_fitlering'),
    kstep(3, val=5, sub=1, desc='library_fitlering'),
    kstep(3, val=5, sub=2, desc='library_fitlering'),
    kstep(3, val=5, sub=3, desc='library_fitlering'),
    kstep(3, val=5, sub=4, desc='library_fitlering'),
    
    kstep(4, val=5, desc='merge_data'),
    kstep(5, val=5, sub=0, desc='filter_data'),
    kstep(5, val=5, sub=1, desc='filter_data'),
    kstep(5, val=5, sub=2, desc='filter_data'),
    kstep(5, val=5, sub=3, desc='filter_data'),
    kstep(5, val=5, sub=4, desc='filter_data'),
    kstep(5, val=5, sub=5, desc='filter_data'),
    
    kstep(6, val=20, desc='embed'),
    kstep(7, desc='save'),
]

REQUIRED_FILES = [
    'df_counts.pkl', 'df_phate.pkl', 
    'df_pca.pkl', 'phate_op.pkl'
]

REQUIRED_ATTRS = [
    'df_counts', 'df_phate', 'df_pca',
    'phate_op', 'pca_loadings', 'pca_components'
]

## DataModule

In [ ]:
#| export
@dataclass
class EmbryoidBodies2018DataModule(pl.LightningDataModule):
    time_col: str = SAMPLES
    data_dir: str = os.path.expanduser('~/Downloads/embryoid_2018')
    batch_size: int = 32
    use_time_dataset:bool = True

    primary: str = 'counts'
    dl_kwargs: Dict[str, Any] = field(default_factory=dict)

    _: KW_ONLY
    perc_train: float = 0.7
    perc_valid: float = 0.1
    perc_test: float = 0.2

    def get_primary_df(self):
        match self.primary:
            case 'counts':
                return self.df_counts
            case 'phate':
                return self.df_phate
            case 'pca':
                return self.df_pca
            case _:
                raise ValueError

    @property
    def pbar(self):
        try:
            return self._pbar
        except AttributeError:
            self._pbar = kuut(steps=STEPS, desc='EB Loader')
            self._pbar.display()
            return self._pbar
    
    @property
    def sc_rna_dir(self):
        return os.path.join(self.data_dir, SINGLE_CELL_SUBDIR)
    
    @property
    def timepoint_dirs(self):
        return [os.path.join(self.sc_rna_dir, d) for d in TIMEPOINT_DIRS]
    
    @property
    def has_all_timepoint_dirs(self):
        return all(os.path.exists(d) for d in self.timepoint_dirs)
    
    @property
    def does_data_dir_exists(self):
        return os.path.isdir(self.data_dir)

    @property
    def does_sc_rna_dir_exists(self):
        return os.path.isdir(self.sc_rna_dir)
    
    @property
    def has_all_saved_files(self):
        try:
            files = os.listdir(self.data_dir)
            return all(list(map(lambda e: e in files, REQUIRED_FILES)))  
              
        except FileNotFoundError:
            return False
        
    @property
    def is_data_downloaded(self):
        return self.does_sc_rna_dir_exists and self.has_all_saved_files
        
    def make_data_dir(self):        
        make_missing_dirs([self.data_dir])

    def make_sc_rna_dir(self):        
        make_missing_dirs([self.sc_rna_dir])

    
    def download(self):
        self.pbar.step(0)
        if self.has_all_saved_files:
            pass
        
        if not self.has_all_timepoint_dirs:
            self.make_sc_rna_dir()
            self.pbar.write(f'Could not find eb data at {self.data_dir}. Downloading...')
            r = requests.get(MENDELY_URI)
            z = zipfile.ZipFile(io.BytesIO(r.content))
            z.extractall(self.data_dir)

        self.pbar.step(1)

    def _load_timepoint(self, t:str, sparse:bool=True):
        return scprep.io.load_10X(os.path.join(self.sc_rna_dir, t), sparse=sparse, gene_labels='both')
    
    def load_timepoints(self, sparse:bool=True):
        self.raw = {}
        self.pbar.write(f'Loading 10X genomics data')
        for i, t in enumerate(TIMEPOINT_DIRS):            
            self.raw[i] = self._load_timepoint(t, sparse)
            self.pbar.step(2, i)
            
        self.pbar.step(2)
        return self.raw
    
    def library_fitlering(self):
        self.filtered = []
        self.pbar.write(f'Filtering library size')
        for i, (t, batch) in enumerate(self.raw.items()):
            batch = scprep.filter.filter_library_size(batch, percentile=20, keep_cells='above')
            batch = scprep.filter.filter_library_size(batch, percentile=75, keep_cells='below')
            self.filtered.append(batch)
            self.pbar.step(3, i)
            
        del self.raw
        self.raw = None
        self.pbar.step(3)
        return self.filtered
    
    def merge_data(self):
        self.df_counts, self.sample_labels = scprep.utils.combine_batches(
            self.filtered, HUMAN_FRIENDLY_TIMEPOINTS, append_to_cell_names=True
        )
        del self.filtered
        self.pbar.step(4)        
        return self.df_counts
    
    def filter_data(self):
        self.pbar.write('Filtering genes and normalizing')
        self.df_counts = scprep.filter.filter_rare_genes(self.df_counts, min_cells=10)
        self.pbar.step(5, 0)        

        self.df_counts = scprep.normalize.library_size_normalize(self.df_counts)
        self.pbar.step(5, 1)        

        # Get all mitochondrial genes. There are 14, FYI.
        mito_genes = scprep.select.get_gene_set(self.df_counts, starts_with="MT-")
        self.pbar.step(5, 2)        

        self.df_counts, self.sample_labels = scprep.filter.filter_gene_set_expression(
            self.df_counts, self.sample_labels, genes=mito_genes, 
            percentile=90, keep_cells='below'
        )
        self.pbar.step(5, 3)
        
        self.df_counts = scprep.transform.sqrt(self.df_counts)
        self.pbar.step(5, 4)
        
        self.df_counts = self.df_counts.join(self.sample_labels)
        self.df_counts = self.df_counts.rename({'sample_labels': self.time_col}, axis=1)        
        self.df_counts.columns = self.df_counts.columns.map(lambda e: e.split(' ')[0])
        self.pbar.step(5, 5)
        
        self.pbar.step(5)

    def embed(self):
        self.pbar.write('Embedding data')
        self.phate_op = phate.PHATE(n_jobs=-2, random_state=42)
        self.Y_phate = self.phate_op.fit_transform(
            self.df_counts.drop(columns=self.time_col)
        )

        self.df_phate = pd.DataFrame(
            np.hstack((
                self.Y_phate, 
                self.sample_labels.values.reshape(-1, 1)
            )), index=self.df_counts.index, columns=f'P1 P2 {self.time_col}'.split()
        )
        
        self.pca_loadings = self.phate_op.graph.data_nu
        self.pca_components = self.phate_op.graph.data_pca.components_
        self.df_pca = pd.DataFrame(
            self.pca_loadings, 
            columns=[f'd{i}' for i in range(1, 101)],
        )
        self.df_pca.loc[:, self.time_col] = pd.Series(
            self.sample_labels.values, name=self.time_col
        )
        
        self.pbar.step(6)
        
        return self.df_phate
    
    def save(self):
        self.pbar.write('Saving')
        self.make_data_dir()
        self.df_counts.to_pickle(os.path.join(self.data_dir, 'df_counts.pkl'))
        self.df_phate.to_pickle(os.path.join(self.data_dir, 'df_phate.pkl'))
        self.df_pca.to_pickle(os.path.join(self.data_dir, 'df_pca.pkl'))
        with open(os.path.join(self.data_dir, 'phate_op.pkl'), 'wb') as f:
            pickle.dump(self.phate_op, f)
            
        self.pbar.step(7)


    def pipeline(self):
        self.load_timepoints()
        self.library_fitlering()
        self.merge_data()
        self.filter_data()
        self.embed()

    def prepare_data(self):
        '''
        Notes
        -----
        - prepare_data is called from the main process. It is not recommended 
            to assign state here (e.g. self.x = y) since it is called on a 
            single process and if you assign states here then they won’t be 
            available for other processes.
        '''
        # download
        self.download()


    @property
    def has_all_attrs(self):        
        return all(list(map(lambda a: hasattr(self, a), REQUIRED_ATTRS)))


    def load_or_download(self):
        if self.has_all_saved_files:
            self.load()
        else:
            self.download()


    def load(self):
        self.pbar.write('Loading data')
        
        if getattr(self, 'df_counts', None) is None:
            self.df_counts = pd.read_pickle(os.path.join(self.data_dir, 'df_counts.pkl'))
            
        if getattr(self, 'df_cdf_phateounts', None) is None:
            self.df_phate = pd.read_pickle(os.path.join(self.data_dir, 'df_phate.pkl'))
            
        if getattr(self, 'df_pca', None) is None:
            self.df_pca = pd.read_pickle(os.path.join(self.data_dir, 'df_pca.pkl'))

        if getattr(self, 'phate_op', None) is None:
            with open(os.path.join(self.data_dir, 'phate_op.pkl'), 'rb') as f:
                self.phate_op = pickle.load(f)    
                
        if getattr(self, 'pca_loadings', None) is None:
            self.pca_loadings = self.phate_op.graph.data_nu
            
        if getattr(self, 'pca_components', None) is None:
            self.pca_components = self.phate_op.graph.data_pca.components_
            
        self.pbar.step(7)
        self.pbar.write('Data ready!')


    def setup(self, stage: Optional[str]=None):
        # load
        self.load_or_download()
        
        # filtering
        if not self.has_all_attrs:
            self.pipeline()
            self.save()
            
        idxs_train, idxs_valid, idxs_test = random_split_dataframe(
            self.get_primary_df().drop(columns=self.time_col),
            [self.perc_train, self.perc_valid, self.perc_test],
            as_dataframes=False
        )
        self.idxs_train = idxs_train
        self.idxs_valid = idxs_valid
        self.idxs_test = idxs_test
            
        if stage == 'fit':
            pass
        
        if stage == 'test':
            pass
        
        if stage == 'predict':
            pass


    def make_dataset(self, df: DataFrame):
        if self.use_time_dataset:
            ds = TimeDataset(df, time_key=self.time_col)        
        else:
            ds = DataFrameDataset(df, label_col=self.time_col)
        return ds
    
    def make_dataloader(self, ds):
        # NOTE: makes sure batch_size overwrites values in dl_kwargs
        kwargs = dict(**self.dl_kwargs, **dict(batch_size=self.batch_size, drop_last=True))
        dl = DataLoader(ds, **kwargs)
        return dl

    def train_dataloader(self):
        df_sub = self.get_primary_df().iloc[self.idxs_train]
        ds = self.make_dataset(df_sub)
        self.train_ds = ds
        return self.make_dataloader(ds)
        

    def val_dataloader(self):
        df_sub = self.get_primary_df().iloc[self.idxs_valid]
        ds = self.make_dataset(df_sub)
        self.valid_ds = ds
        return self.make_dataloader(ds)
        
    def test_dataloader(self):
        df_sub = self.get_primary_df().iloc[self.idxs_test]
        ds = self.make_dataset(df_sub)
        self.test_ds = ds
        return self.make_dataloader(ds)
    
    def predict_dataloader(self):
        '''
        See Also
        --------
        https://github.com/Lightning-AI/lightning/discussions/11455#discussioncomment-1959264
        
        Notes
        -----        
        Test: is used for a holdout section of your dataset, used to evaluate a model after 
            you've done hyperparameter tuning for fair comparison. The test set has labels 
            included
            
        Prediction: There are no labels available, only the model inputs, meaning prediction isn't 
            used for evaluation

        Regarding differences in Lightning: the two code paths are pretty similar are very similar. 
            The model is put into eval mode, gradients are disabled, and the trainer makes one 
            pass through the corresponding dataloader(s), calling the relevant hooks in the 
            Lightning Module or callback (prefixed with test or predict)
        '''
        return self.test_dataloader()

    def teardown(self, stage:str):
        pass

In [ ]:
#| eval: False
eb = EmbryoidBodies2018DataModule(
    primary='pca',
    batch_size=8,
)

In [ ]:
#| eval: False
eb.does_data_dir_exists, eb.does_sc_rna_dir_exists, eb.has_all_timepoint_dirs

(True, True, True)

In [ ]:
#| eval: False
eb.setup()

EB Loader:   0%|          | 0/126 [00:00<?, ?it/s]

Loading data
Data ready!


In [ ]:
#| eval: False
dl = eb.train_dataloader()

In [ ]:
#| eval: False
eb.train_ds.df[eb.train_ds.df.columns[-5:]].head()

,d97,d98,d99,d100,samples
5100,0.881895,-1.463558,0.309755,-0.156539,Day 06-09
15125,1.099650,-1.794608,-1.442124,-0.621449,Day 24-27
9431,-1.441780,0.068481,2.492943,2.289345,Day 12-15
13078,-0.047084,0.543089,1.502882,1.567946,Day 18-21
3193,0.652287,0.758303,0.681693,-0.214407,Day 06-09


In [ ]:
#| eval: False
eb.train_ds.t_min, eb.train_ds.t_max

(0, 4)

In [ ]:
#| eval: False
for x, y in dl:
    break
x.shape, y.shape

(torch.Size([8, 5, 100]), torch.Size([8, 5]))

In [ ]:
#| eval: False
for x, y in eb.train_ds:
    break
x.shape, y.shape

(torch.Size([5, 100]), torch.Size([5]))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()